In [2]:
import pathlib, json

data = json.loads(pathlib.Path("/scr/ig_pipeline/metadata/avg_category_specs.json").read_text())

In [3]:
data

{'ceilings': {'mass': 1000.0,
  'volume': 150.5171386627509,
  'density': 6.64376169308269},
 'walls': {'mass': 15000.0,
  'volume': 8.527721345183009,
  'density': 1758.9692947076583},
 'floors': {'mass': 500.0,
  'volume': 22.233914914393782,
  'density': 22.488167375162085},
 'armchair': {'mass': 20.0,
  'volume': 0.11771597793558022,
  'density': 169.9004701888893},
 'bathtub': {'mass': 110.0,
  'volume': 0.279054958718872,
  'density': 394.1875840694778},
 'bed': {'mass': 50.0,
  'volume': 1.431610449027443,
  'density': 34.92570205391225},
 'bench': {'mass': 25.0,
  'volume': 0.8418784199161071,
  'density': 29.695499265192282},
 'bottom_cabinet': {'mass': 50.0,
  'volume': 0.04522725043785853,
  'density': 1105.5281830298118},
 'bottom_cabinet_no_top': {'mass': 40.0,
  'volume': 0.05575634862501401,
  'density': 717.4070932983365},
 'breakfast_table': {'mass': 18.0,
  'volume': 0.12208614849896396,
  'density': 147.43687323507262},
 'burner': {'mass': 0.5,
  'volume': 0.01525250

In [8]:
from bddl.knowledge_base import *

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to /home/cgokmen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Complained object upmkjr does not exist in the database. Skipping.
Complained object wmchsj does not exist in the database. Skipping.
Complained object uqusgj does not exist in the database. Skipping.
Complained object aginyz does not exist in the database. Skipping.
Complained object fzisrr does not exist in the database. Skipping.
Complained object pcpvvu does not exist in the database. Skipping.
Complained object qbxmgb does not exist in the database. Skipping.
Complained object qbxmgb does not exist in the database. Skipping.
Complained object adimyi does not exist in the database. Skipping.
Complained object adimyi does not exist in the database. Skipping.
Complained object csxnnw does not exist in the database. Skipping.
Complained object ewbqsp does not exist in the database. Skipping.
Complained ob

In [12]:
# Load particle params and delete everything that shows up there.
import csv
synset_has_particle_density = set()
with open("/scr/ig_pipeline/metadata/substance_hyperparams.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        synset = Synset.get(row["synset"].strip())
        if synset is None:
            print(f"Synset {row['synset']} not found")
            continue
        has_density = "particle_density" in json.loads(row["hyperparams"])
        if has_density:
            synset_has_particle_density.add(synset)
        elif "visualSubstance" not in synset.property_names:
            print(f"Synset {synset} does not have particle density")

synset_is_visual = {s for s in Synset.all_objects() if "visualSubstance" in s.property_names}
ignore_synsets = synset_has_particle_density | synset_is_visual
ignore_categories = {c for s in ignore_synsets for c in s.categories}

Synset diced__ivy.n.01 not found


In [13]:
densities = {Category.get(k): v["density"] for k, v in data.items() if Category.get(k) not in ignore_categories}
keys, values = zip(*sorted(densities.items(), key=lambda x: -x[1]))

In [14]:
# Print the first 100 values
for i in range(100):
    print(f"{keys[i]}: {values[i]}")

heap_of_gravel: 142972136.67771518
half_french_fries: 65148551.62416501
half_agave: 55408913.45224245
half_broccoli: 43021618.0417507
half_gelatin: 38834277.25682409
half_brisket: 37346889.86641628
half_broccoli_rabe: 16738221.833604608
half_venison: 12622545.958835993
half_butter: 9189055.761762336
half_broccolini: 6448298.366695111
half_veal: 4872613.252453373
plywood: 4780403.971961479
half_bean_curd: 4243621.766550625
half_bleu: 2957505.3340678574
half_garlic_clove: 2544506.8163454793
half_date: 2449266.730369535
half_gingerbread: 2029325.8517836337
half_turkey_leg: 1558066.9855943858
half_dried_apricot: 1408523.3173061723
moth_ball: 1357739.8792303982
half_durian: 1002299.1985675347
half_turkey: 993774.0272999855
half_fennel: 978836.1556186783
half_bok_choy: 963000.2755115384
half_ginger_root: 834158.381180437
diamond: 816944.4023636346
half_beet: 813868.6408547036
half_fillet: 720225.1199537076
half_burrito: 670186.4618204486
ruby: 661840.7753177595
half_enchilada: 648069.5187083

In [17]:
# Print the last 100 values
for i in range(100):
    print(f"{keys[-i-1]}: {values[-i-1]}")

background: 0.010818143452382517
rail_fence: 0.24735533276111102
greenery: 0.3073414535797648
balloon: 1.0037331641881133
copper_wire: 1.2717878579773034
garden_plant: 1.4387115805277007
tablecloth: 1.6273356019436962
curtain: 1.704200375259002
roof: 2.4922133185054895
volleyball_net: 2.740411949659848
tights: 3.0419958497790414
dress_shirt: 3.2438586469742146
information_bulletin: 3.26430398623506
plastic_bag: 3.6092460012862624
bush: 4.0102565047411405
diaper: 4.1069771705174345
toy_car: 4.62598966740203
vine: 4.8123553624826885
pottable_beefsteak_tomato: 4.941305381161639
fixed_window: 5.089489905128521
garden_umbrella: 5.541384350658249
room_light: 5.5556014823040245
sauna_suit: 5.802330558118299
window_blind: 6.408855867301658
garden_statue: 6.495631900052453
ceilings: 6.64376169308269
hammam_bench: 6.752377872658571
onesie: 7.086326159421505
popcorn_bag: 7.162331632277959
pillowcase: 7.276251631560152
legging: 7.656409819139374
skeletal_frame: 7.973782928600498
hoodie: 7.97598627

In [ ]:
# How many categories are heavier than 2,000 kg/m^3? (e.g. concrete)
heavy_categories = [c for c, d in densities.items() if d > 2000]
print(f"Number of categories heavier than 2,000 kg/m^3: {len(heavy_categories)}")

# How many categories are lighter than 1 kg/m^3? (e.g. air)
light_categories = [c for c, d in densities.items() if d < 1]
print(f"Number of categories lighter than 1 kg/m^3: {len(light_categories)}")

Number of categories heavier than 10,000 kg/m^3: 447
Number of categories lighter than 1 kg/m^3: 3
